In [108]:
# 모두닥 크롤링 모듈

In [8]:
import sys

sys.version

'3.8.13 (default, Mar 28 2022, 06:59:08) [MSC v.1916 64 bit (AMD64)]'

In [9]:
!pip install selenium

Defaulting to user installation because normal site-packages is not writeable


In [50]:
import pandas as pd    # csv
from selenium.webdriver.chrome.options import *
from bs4 import *
import time
import re
import base64    # encoder
# webdriver = 웹페이지를 제어하기 위한 모듈 >> 웹 그자체가 됨!
from selenium import webdriver as wb
# Keys = 컴퓨터의 키보드와 같은 역할 모듈, 컴퓨터가 쓸 키보드 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [13]:
# 버전 체크

print(wb.__version__)

4.3.0


In [20]:
# 병원 분과 페이지에서 총 몇 페이지까지 있는지 구하는 함수
def get_total_page():
    move_to_last_page()
    total_page = get_page_count()
    driver.back()
    time.sleep(1)
    
    return total_page


# 끝 페이지 이동하는 함수
def move_to_last_page():
    driver.find_element('xpath', '/html/body/div[3]/div/div/div[1]/div[3]/nav/ul/li[4]/a').click()
    time.sleep(1)

    
# 현재 페이지 몇인지 구하는 함수
def get_page_count():
    page_count = []
    
    for text in driver.current_url[::-1]:
        if text == '=':
            break
        page_count.append(text)
    page_count.reverse()
    page_count = int(''.join(page_count))
    
    return page_count


In [157]:
# 병원 리스트들 뜨는 페이지에서 작업

# 현재 페이지 내 병원 박스 가져오는 함수
def get_hospital_boxes():
    element_boxes = driver.find_elements("xpath", "//div[@class='doctor-total-box border-bottom']")
    
    return element_boxes


# # 병원의 리뷰 페이지 수 계산
# def get_num_review_page(element_box):
#     num_review = get_review_count(element_box)
#     num_review_page = (num_review // 10) + 1

#     return num_review_page

# 한 페이지에서 병원 url들과 리뷰 갯수 리스트로 가져오는 함수
def get_list_url_and_num_review_page(hospital_boxes):
    list_data = []
    for hospital_box in hospital_boxes:
        data = [get_hospital_url(hospital_box), get_num_review_page(hospital_box)]
        list_data.append(data)
        
    return list_data


# 박스에서 리뷰 갯수 가져오는 함수
def get_num_review_page(element_box):
    review_count = element_box.find_element(By.CLASS_NAME, "hospitals-review-count").text
    review_count = review_count[3:]
    review_count = int(review_count)
    
    num_review_page = (review_count // 10) + 1
    
    return num_review_page


# 병원 url 가져오는 함수
def get_hospital_url(element_box):
    temp = element_box.find_element(By.TAG_NAME, "a")
    hospital_url = temp.get_attribute("href")
    
    return hospital_url


In [140]:
def get_hospital_name():
    hospital_name = driver.find_element(By.CLASS_NAME, "hospital-doctor-name-box").text
    
    return hospital_name


# 리뷰 박스들 가져오는 함수
def get_review_boxes():
    element_boxes = driver.find_elements("xpath", "//div[@class='review-in-detail-text-box-content col-8 pl-5']")
    
    return element_boxes


# 리뷰 박스에서 받은 치료 가져오는 함수
def get_therapy(element_box):
    therapy = element_box.find_element(By.CLASS_NAME, "title-box").text
    therapy = therapy[8:]
    
    return therapy


# 리뷰 박스에서 리뷰 텍스트 가져오는 함수
def get_review(element_box):
    review = element_box.find_element(By.CLASS_NAME, "content-box").text
    review = clean_text(review)

    return review


# 해당 페이지로 넘어가는 함수
def move_page(current_url, page):
    page_url = current_url +  "?page=" + str(page)
    driver.get(page_url)

In [109]:
element_boxes = get_review_boxes()
print(get_review(element_boxes[13]))

저는 여기 너무너무 추천합니다
1. 실력 보장해요
제가 다른 치과 두군데서나 발견하지 못했는데 알고보니 심각했던 신경치료해야 하는 치아를 이분께서 유일하게 발견해주셨습니다😭😭 생각보다 너무 심각해서 결국 대학병원에 가서 치료받기로 했지만, 여기 아니었으면ㅠㅠ 치료해야 한다는 생각도 못한 채 방치했을 거에요ㅠㅠ 아찔했습니다ㅠㅠ 저도 모르는 새에 뼈가 녹고 있었거든요,,, 이래서 진짜 실력있는 곳을 가야 하는구나 생각했급니다!!! 정말 선생님 다시 한번 너무 감사드립니다!!!
2. 과잉진료 없음
충치가 조금 많았는데 다른 곳은 다 치료하라고 비싼 진료비를 요구했지만(그래놓고 정작 심각한 치아는 못알아보심) 여기서는 딱 필요한 치아만 치료하고 계속 경과 지켜보자고 하셨습니다!
3. 충치치료도 만족스러웠습니다.
깔끔하고 필요한 것만 설명해주시고 이해가 잘되게 얘기해주셔요! 앞으로도 계속 다닐 예정입니다!


In [127]:
# 로그인
def login(id, password):
    driver.find_element('xpath', '/html/body/nav/div/div/ul[2]/ul/li[3]/a').click() #id 속성으로 접근

    user_email = driver.find_element('xpath', '/html/body/div[12]/div/div/div[3]/form/input[1]')
    user_email.send_keys(id)
    user_password = driver.find_element('xpath', '/html/body/div[12]/div/div/div[3]/form/input[2]')
    user_password.send_keys(password)

    # 로그인 버튼 클릭
    driver.find_element('xpath', '/html/body/div[12]/div/div/div[3]/form/button').click()
    time.sleep(3)

    
# 병원의 리뷰 페이지 수 계산
def count_page():
    num_review = driver.find_element('xpath', '/html/body/div[3]/div/div[2]/div[2]/div/div[1]/div[1]').text
        
    num_review = int(num_review[7:])

    num_page = (num_review // 10) + 1

    return num_page


# # 병원의 리뷰 페이지 수 계산(병원 그림 없는경우)
# def count_page_except():
#     num_review = driver.find_element('xpath', '/html/body/div[3]/div/div[1]/div[2]/div/div[1]/div[1]').text
        
#     num_review = int(num_review[7:])

#     num_page = (num_review // 10) + 1

#     return num_page


# 텍스트 전처리
def clean_text(text):
#     cleaned_text = re.sub('[a-zA-Z]' , '', text)
#     cleaned_text = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]','', cleaned_text)
    cleaned_text = re.sub('[\n]', ' ', text)
    
    return cleaned_text

# def clean_str(text):
#     cleaned_text = re.sub('[a-zA-Z]' , '', text)
#     cleaned_text = re.sub('[\{\}\[\]\/?,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]','', cleaned_text)
#     cleaned_text = re.sub('[\n]', ' ', cleaned_text)
    
#     pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
#     cleaned_text = re.sub(pattern=pattern, repl='', string=cleaned_text)
#     pattern = '[^\w\s]'         # 특수기호제거
#     cleaned_text = re.sub(pattern=pattern, repl='', string=cleaned_text)
#     return cleaned_text  



# 병원 페이지로 이동
def move_to_hospital(hospital):
    search_box = driver.find_element('xpath', '/html/body/div[2]/div/div[3]/form/div/div/span/input[2]')
    search_box.send_keys(hospital)
    driver.find_element('xpath', '/html/body/div[2]/div/div[3]/form/div/div/div/button/img').click()
    time.sleep(1)

    for i in range(10):
        try:
            element = driver.find_element('xpath', f'/html/body/div[3]/div/div/div[1]/div[3]/div[{i}]/a/div/div[2]/div[2]/div[1]/div[1]/div[1]')

            if element.text[:3] == hospital[:3]:
                element.click()
                time.sleep(1)
                break

        except:
            pass

In [15]:
# 병원 분과 종류
hospital_kind_list = ['치과', '피부과', '성형외과', '산부인과', '정신건강의학과', '비뇨기과','재활의학과', '외과', '소아과', '가정의학과', '한의원']

# 모두닥 병원 접근 쿼리
hospital_kind_query = "https://www.modoodoc.com/hospitals/?search_query="


In [5]:
# 모두닥 페이지 접속 후 로그인

In [18]:
# 모두닥 페이지 url
modoodoc_url = "https://www.modoodoc.com/"

## 크롬 드라이버 실행
#  wb.Chrome() = 컴퓨터가 크롬창을 씀
driver = wb.Chrome()

# 모두닥 페이지 이동
driver.get(modoodoc_url)

# 로그인
id = 'random1234@random.com'
password = 'random1234'
login(id, password)

In [158]:
hospital_boxes = get_hospital_boxes()

list_url_and_num_review = get_list_url_and_num_review(hospital_boxes)

for url, num_review in list_url_and_num_review:
    print(num_review)

9
4
5
5
4
4
4
4
4
4


In [159]:
data = []
data_unit = []

hospital_boxes = get_hospital_boxes()
list_url_and_num_review_page = get_list_url_and_num_review_page(hospital_boxes)

for url, num_review_page in list_url_and_num_review_page:
    driver.get(url)
    time.sleep(1)
    
    current_url = driver.current_url
   
    for page in range(1, num_review_page+1):
        move_page(current_url, page)
        time.sleep(1)
        
        hospital_name = get_hospital_name()
        review_boxes = get_review_boxes()
    
        for review_box in review_boxes:
            try:
                therapy = get_therapy(review_box)
                review = get_review(review_box)

                data_unit = [hospital_name, therapy, review]
                data.append(data_unit)

            except:
                pass

In [160]:
data

[['자생한방병원',
  '추나요법',
  '척추디스크 시술후 지속적인 치료가 필요하여 방문했습니다. 다른곳들과는 차별화되게 체계적인 시스템이 좋아요. 이동 시간이 조금소요되지만 친절하게 상담해주시고 꼼꼼하게 치료해주셔서좋았습니다.'],
 ['자생한방병원',
  '침술 , 부항 , 한약 , 약침 , 전기침',
  '어깨 석회화건염으로 어깨,머리,팔,등 전체까지 통증이 있던터라 어깨 돌리는 것도 여유치않았던 상황이였습니다. 원장님께서 신체 검진 하신 뒤 침,약침,부항 치료를 해주시면서 어깨 가동범위가 잘 나오더라구요 . 혈자리도 마킹하여 알려주시면서 수시로 자극하라고도 알려주시면서 매우 친절하게 설명해주셨습니다.'],
 ['자생한방병원',
  '침술 , 한약',
  '매우 친절하세요~~ 질문이 정말 많았는데 친절하게 자세히 다 답해주시고 입원해있는 기간동안 치료 잘 받았습니다… 매우 만족스러웠어요!! 앞으로 치료 받을일 있으면 여기로 또 올거에요!'],
 ['자생한방병원',
  '침술 , 한약 , 추나요법 , 약침 , 전기침 , 뜸',
  '약 10년째 믿고 계속 다니고 있는 병원입니다 병원 시설도 너무 잘 되어있고 깨끗하고 치료실도 너무 좋고 치료 효과도 너무 좋아요 증상도 많이 나아졌고 앞으로도 꾸준히 다닐 예정입니다'],
 ['자생한방병원',
  '추나요법 , 한방 온열치료 , 전기침',
  '교통사고 나서 갔어요 집 앞이가도 했고 자생한방병원이 교통사고 치료로 유명해서요 체계적이였고 의사선생님 직원분들 다 친절하셨어요 건물도 깨끗해서 다니기 좋았어요 물리치료실도 넓고 옷 갈아입는 거 하나하나 잘 가려주셔서 감사했어요'],
 ['자생한방병원',
  '침술 , 한방 온열치료 , 전기침',
  '처음 방문시 절차를 상세하게 설명해주시면서 대화 나누기 편했습니다. 큰병원이라 걱정했었는데요. 친절하게 설명해주신덕에 가는게 편안했어요. 허리통증이어서 자주가야되는 상황이었는데, 치료 잘받았어요'],
 ['자생한방병원',
  '침술 , 추나요법 , 약침',
  '허리협

In [ ]:
# 맨 위 병원그림 있는 경우

# 크롤링 후 리스트 삽입

data = []
data_unit = []

for index in range(35, len(hospital_list)):
    # 병원 이름 검색 통해 접근(리스트 불러와서 넣는 것으로 변경 예정)
    hospital = hospital_list[index]
    move_to_hospital(hospital)
    
    current_url = driver.current_url
    num_page = count_page()
    
    for page in range(1, num_page+1):
        url = current_url + f"?page={page}"
        driver.get(url)
        time.sleep(2)

        for j in range(2, 16):
            try:
                # 병원 이름 크롤링
                hospital = driver.find_element('xpath', '/html/body/div[3]/div/div[2]/div[1]/h1').text

                # 받은 치료 이름 크롤링
                treatment = driver.find_element('xpath', f'/html/body/div[3]/div/div[4]/div[1]/div[4]/div[5]/div[{j}]/div/div[2]/div[2]').text


                # 받은 치료 명시 안된 부분 처리
                if treatment[0] != '받':
                    continue
                treatment = treatment[8:]

                # 리뷰 내용 크롤링
                review = driver.find_element('xpath', f'/html/body/div[3]/div/div[4]/div[1]/div[4]/div[5]/div[{j}]/div/div[2]/div[3]').text
                # 리뷰 텍스트 전처리
                review = clean_text(review)

                # 크롤링 데이터 리스트 삽입
                data_unit = [hospital, treatment, review]
                data.append(data_unit)

            except:
                pass
    
    
    
    driver.get(modoodoc_url)
    time.sleep(3)

In [8]:
# # 읽어올 엑셀 파일 지정
# filename = 'temp.xlsx'

# # 엑셀 파일 읽어 오기
# data = pd.read_excel(filename, engine='openpyxl',  )
# hospital_list = data.병원이름.to_list()

In [229]:
# 맨 위 병원그림 있는 경우

# 크롤링 후 리스트 삽입

data = []
data_unit = []

for index in range(35, len(hospital_list)):
    # 병원 이름 검색 통해 접근(리스트 불러와서 넣는 것으로 변경 예정)
    hospital = hospital_list[index]
    move_to_hospital(hospital)
    
    current_url = driver.current_url
    num_page = count_page()
    
    for page in range(1, num_page+1):
        url = current_url + f"?page={page}"
        driver.get(url)
        time.sleep(2)

        for j in range(2, 16):
            try:
                # 병원 이름 크롤링
                hospital = driver.find_element('xpath', '/html/body/div[3]/div/div[2]/div[1]/h1').text

                # 받은 치료 이름 크롤링
                treatment = driver.find_element('xpath', f'/html/body/div[3]/div/div[4]/div[1]/div[4]/div[5]/div[{j}]/div/div[2]/div[2]').text


                # 받은 치료 명시 안된 부분 처리
                if treatment[0] != '받':
                    continue
                treatment = treatment[8:]

                # 리뷰 내용 크롤링
                review = driver.find_element('xpath', f'/html/body/div[3]/div/div[4]/div[1]/div[4]/div[5]/div[{j}]/div/div[2]/div[3]').text
                # 리뷰 텍스트 전처리
                review = clean_text(review)

                # 크롤링 데이터 리스트 삽입
                data_unit = [hospital, treatment, review]
                data.append(data_unit)

            except:
                pass
    
    
    
    driver.get(modoodoc_url)
    time.sleep(3)

In [239]:
# 맨 위 병원그림 없는 경우

# 크롤링 후 리스트 삽입

data = []
data_unit = []

for index in range(len(hospital_list)):
    # 병원 이름 검색 통해 접근(리스트 불러와서 넣는 것으로 변경 예정)
    hospital = hospital_list[index]
    move_to_hospital(hospital)
    
    current_url = driver.current_url
    num_page = count_page_except()
    
    for page in range(1, num_page+1):
        url = current_url + f"?page={page}"
        driver.get(url)
        time.sleep(2)

        for j in range(2, 16):
            try:
                # 병원 이름 크롤링
                hospital = driver.find_element('xpath', '/html/body/div[3]/div/div[1]/div[1]/h1').text

                # 받은 치료 이름 크롤링
                treatment = driver.find_element('xpath', f'/html/body/div[3]/div/div[3]/div[1]/div[4]/div[5]/div[{j}]/div/div[2]/div[2]').text
                                                         
                # 받은 치료 명시 안된 부분 처리
                if treatment[0] != '받':
                    continue
                treatment = treatment[8:]

                # 리뷰 내용 크롤링
                review = driver.find_element('xpath', f'/html/body/div[3]/div/div[3]/div[1]/div[4]/div[5]/div[{j}]/div/div[2]/div[3]').text
                # 리뷰 텍스트 전처리
                review = clean_text(review)
                                                       
                # 크롤링 데이터 리스트 삽입
                data_unit = [hospital, treatment, review]
                data.append(data_unit)

            except:
                pass
    
    
    
    driver.get(modoodoc_url)
    time.sleep(3)

In [240]:
# 데이터 프레임 생성 및 데이터 입력
df = pd.DataFrame(columns = ['병원 이름', '받은 치료', '리뷰'])

num = len(data)

for i in range(num):
    df.loc[i]=data[i]
   
df

,병원 이름,받은 치료,리뷰
0,노원CNP차앤박피부과,여드름 압출,제 여드름상태에 대해서 자세히 알고싶었으나 너무 획일화된듯한 설명이 조금 아쉬웠습니...
1,노원CNP차앤박피부과,"미간보톡스 , 여드름 압출",한줄평 노원에서 괜찮다고 생각하는 피부과 친절하시지만 사람많은병원답게 대기시간은 ...
2,노원CNP차앤박피부과,피부관리,1 송원장님 진료보려했는데 대기가 길어 유원장님 진료봤고 다른 피부과 의사에 비해 ...
3,노원CNP차앤박피부과,여드름 압출,한줄평 눈 뜨고 코베어가는 병원 다신 안간다 뾰루지 하나 짜러왔다 이야기했더니 염...
4,노원CNP차앤박피부과,트러블,한줄평 노원역 무난한 피부과 피부에 오돌토돌한게 나서 가게 됐어요 피부과는 거의 ...
5,노원CNP차앤박피부과,피부염,한줄평 처방은 스테로이드라 아쉽지만 의사선생님은 친절하셨어요 클렌징 오일 사용과 ...
6,노원CNP차앤박피부과,피부염,한줄평 인기많은 피부과 진료실에 들어서자 친절히 인사해주셨고 증상에 대해 들으시고...
7,노원CNP차앤박피부과,염증치료,한줄평 깨끗 피어싱 염증이 심해서 방문했어요 간단한 진료라 금방 끝났지만 친절하고...
8,노원CNP차앤박피부과,습진,한줄평 친절하나 효과는 잘 모르겠음 원장님 친절하시고 상담 길게 잘해주세요 다만 ...
9,노원CNP차앤박피부과,염증치료,의사선생님과 간호사분들은 모두 친절하시고 병원도 크고 깨끗합니다 치료 후 결과는 경...


In [241]:
# 엑셀 저장
df.to_excel('크롤링결과.xlsx')

In [261]:
# # 폴더에 있는 엑셀 파일 내용 병합

# import os

# file_list = os.listdir('새 폴더')
# file_list

# data_frame = pd.DataFrame(columns = ['병원 이름', '받은 치료', '리뷰'])



# for filename in file_list:
#     tmp =  pd.read_excel('새 폴더/{}'.format(filename))
#     tmp.drop('Unnamed: 0',axis=1,inplace=True)
#     print(tmp)
#     data_frame = pd.concat([data_frame,tmp])
   

        병원 이름           받은 치료  \
0      2D치과의원  치과 스케일링 , 충치치료   
1      2D치과의원            치아교정   
2      2D치과의원            충치치료   
3      2D치과의원            치아교정   
4      2D치과의원           라미네이트   
..        ...             ...   
299  강남레옹치과의원            치아교정   
300  강남레옹치과의원    사랑니발치 , 충치치료   
301  강남레옹치과의원    사랑니발치 , 잇몸치료   
302  강남레옹치과의원           사랑니발치   
303  강남레옹치과의원           사랑니발치   

                                                    리뷰  
0    진짜 핵심만 선생님께서 봐주시고 대부분 치위생사 님이 많이 하시는데 치위생사 님들이...  
1    교정 빠르게 잘 받았어요 앞니 부분 교정이었고 처음엔 45개월 소요 안내받았지만 6...  
2    이벤트 통해서 저렴하게 충치치료 할 수 있었어서 좋았네요 병원 되게 크고 대형 치과...  
3    치위생사분들 데스크 직원 분들 모두 아주 친절하십니다 모든 파트에 설명을 해줘 눈을...  
4    무삭제 라미네이트 광고보고 이 병원을 알게되어 방문했어요 상담받고 라미네이트를 붙이...  
..                                                 ...  
299  항상 나긋한 목소리로 세세하게 설명해주시고 교정담당 선생님이셔서 한달에 한번 이상 ...  
300  저는 네개 다 바로 난 사랑니지만 입안이 좁아 사랑니들이 멀쩡한 이를 밀어낸다고하여...  
301  사랑니가 갑자기 나서 통증을 느껴서 급하게 병원을 찾아보았어요 인터넷으로 레옹치과를...  
302  김영삼 선생님은 사랑니 발치의 신이십니다 인터넷을 

In [263]:
# # 데이터 프레임 인덱스 제거

# data_frame = data_frame.reset_index(drop = True)

# data_frame

,병원 이름,받은 치료,리뷰
0,2D치과의원,"치과 스케일링 , 충치치료",진짜 핵심만 선생님께서 봐주시고 대부분 치위생사 님이 많이 하시는데 치위생사 님들이...
1,2D치과의원,치아교정,교정 빠르게 잘 받았어요 앞니 부분 교정이었고 처음엔 45개월 소요 안내받았지만 6...
2,2D치과의원,충치치료,이벤트 통해서 저렴하게 충치치료 할 수 있었어서 좋았네요 병원 되게 크고 대형 치과...
3,2D치과의원,치아교정,치위생사분들 데스크 직원 분들 모두 아주 친절하십니다 모든 파트에 설명을 해줘 눈을...
4,2D치과의원,라미네이트,무삭제 라미네이트 광고보고 이 병원을 알게되어 방문했어요 상담받고 라미네이트를 붙이...
...,...,...,...
1229,꽃마을한의원,침술,부부치료로 진료다니는중입니다 갈때마다 직원들도 넘 친절하시고 최은미원장님도 진료 잘...
1230,꽃마을한의원,건강검진,대표원장으로 대신등록 꽃마을한방병원에서 거의 매년 건강검진을 받고 있어요 집에서 멀...
1231,꽃마을한의원,디프테리아/파상풍/백일해 예방접종,한줄평 깔끔한 병원 백일해 예방접종 맞기 위해 방문하였습니다 의사선생님과 간단한 ...
1232,꽃마을한의원,한의원 검진,병원이 신식이라 깔끔하고 좋았어요 간호사분들도 친절하시고 의사분들도 이야기 잘들어주...


In [264]:
# # 엑셀 저장
# data_frame.to_excel('크롤링(모두닥).xlsx')

In [118]:
# # 병원 이름 크롤링
# hospital = driver.find_element('xpath', '/html/body/div[3]/div/div[2]/div[1]/h1').text

# # 받은 치료 이름 크롤링
# treatment = driver.find_element('xpath', '/html/body/div[3]/div/div[4]/div[1]/div[4]/div[5]/div[5]/div/div[2]/div[2]').text[8:]

# # 리뷰 내용 크롤링
# review = driver.find_element('xpath', '/html/body/div[3]/div/div[4]/div[1]/div[4]/div[5]/div[5]/div/div[2]/div[3]').text

# # 크롤링 데이터 리스트 삽입
# data_unit = [hospital, treatment, review]
# data.append(data_unit)
# data

In [119]:
# parsing을 위한 함수를 생성

In [120]:
# def remove_html_tags(data):
#     p = re.compile(r'<.*?>')
#     return p.sub(' ', str(data))

# def get_crawl(URL):
#     response = driver.get(URL)
#     html = driver.page_source
#     soup7 = BeautifulSoup(html, 'html.parser')
#     ex_id_divs = soup7.find('div', {'id': 'view_content'})
#     crawl_data = remove_html_tags(ex_id_divs)
#     return crawl_data

In [121]:
# # 리뷰 페이지 이동
# num_page = count_page()

# current_url = driver.current_url

# for i in range(2, num_page+1):
#     url = current_url + f"?page={i}"
#     driver.get(url)
#     time.sleep(1)

In [201]:
# # 병원 이름 검색 통해 접근(리스트 불러와서 넣는 것으로 변경 예정)
# hospital = input()
# search_box = driver.find_element('xpath', '/html/body/div[2]/div/div[3]/form/div/div/span/input[2]')
# search_box.send_keys(hospital)
# driver.find_element('xpath', '/html/body/div[2]/div/div[3]/form/div/div/div/button/img').click()
# time.sleep(1)

# for i in range(10):
#     try:
#         element = driver.find_element('xpath', f'/html/body/div[3]/div/div/div[1]/div[3]/div[{i}]/a/div/div[2]/div[2]/div[1]/div[1]/div[1]')
    
#         if element.text[:2] == hospital[:2]:
#             element.click()
#             break
            
#     except:
#         pass
        
# time.sleep(1)

디에이성형외과


In [9]:
# total_count = 0

# for index in range(len(hospital_list)):
#     hospital = hospital_list[index]
    
#     search_box = driver.find_element('xpath', '/html/body/div[2]/div/div[3]/form/div/div/span/input[2]')
#     search_box.send_keys(hospital)
#     driver.find_element('xpath', '/html/body/div[2]/div/div[3]/form/div/div/div/button/img').click()
#     time.sleep(1)

#     for i in range(10):
#         try:
#             element = driver.find_element('xpath', f'/html/body/div[3]/div/div/div[1]/div[3]/div[{i}]/a/div/div[2]/div[2]/div[1]/div[1]/div[1]')

#             if element.text[:3] == hospital[:3]:
#                 review_count = driver.find_element('xpath', f'/html/body/div[3]/div/div/div[1]/div[3]/div[{i}]/a/div/div[2]/div[2]/div[1]/div[1]/div[2]/div/div[3]').text[3:]
#                 review_count = int(review_count)
#                 total_count += review_count
#                 break

#         except:
#             pass
    
#     driver.get(modoodoc_url)
#     time.sleep(2)
    
# print(total_count)

1514
